In [1]:
import numpy as np
import pandas as pd

import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
import pickle

2023-11-18 16:11:39.575919: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-18 16:11:39.710557: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-18 16:11:39.713386: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-11-18 16:11:39.713399: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [ ]:
# load the embeddings
with open('english_model.pkl') as f:
    eng_embedding = pickle.loads(f)
with open('hindi_model.pkl') as f:
    hin_embedding = pickle.loads(f)

In [2]:
# read the raw files and concatenate
df_train = pd.read_csv('raw_train.csv')
df_test = pd.read_csv('raw_test.csv')

df_final = pd.concat([df_train, df_test])

In [3]:
# store the concatenated files for future use
df_final.to_csv('final.csv', index=False)

In [4]:
# do some further processing
lines=pd.read_csv('final.csv')

# Lowercase all characters
lines['english']=lines['english'].apply(lambda x: x.lower())
lines['hindi']=lines['hindi'].apply(lambda x: x.lower())

# Remove quotes
lines['english']=lines['english'].apply(lambda x: re.sub("'", '', x))
lines['hindi']=lines['hindi'].apply(lambda x: re.sub("'", '', x))

# Remove all the special characters
exclude = set(string.punctuation)
lines['english']=lines['english'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['hindi']=lines['hindi'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines['english']=lines['english'].apply(lambda x: x.translate(remove_digits))
lines['hindi']=lines['hindi'].apply(lambda x: x.translate(remove_digits))

lines['hindi'] = lines['hindi'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
lines['english']=lines['english'].apply(lambda x: x.strip())
lines['hindi']=lines['hindi'].apply(lambda x: x.strip())
lines['english']=lines['english'].apply(lambda x: re.sub(" +", " ", x))
lines['hindi']=lines['hindi'].apply(lambda x: re.sub(" +", " ", x))

# Add start and end tokens to target sequences
lines['hindi'] = lines['hindi'].apply(lambda x : '<sos> '+ x + ' <eos>')

In [5]:
lines.head()

,english,hindi
0,give your application an accessibility workout,<sos> अपने अनुप्रयोग को पहुंचनीयता व्यायाम का ...
1,accerciser accessibility explorer,<sos> एक्सेर्साइसर पहुंचनीयता अन्वेषक <eos>
2,the default plugin layout for the bottom panel,<sos> निचले पटल के लिए डिफोल्ट प्लगइन खाका <eos>
3,the default plugin layout for the top panel,<sos> ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका <eos>
4,a list of plugins that are disabled by default,<sos> उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप ...


In [6]:
# calculate length of tokens and select only those with length less than 20
lines['length_eng_sentence']=lines['english'].apply(lambda x:len(x.split(" ")))
lines['length_hin_sentence']=lines['hindi'].apply(lambda x:len(x.split(" ")))

lines=lines[lines['length_eng_sentence']<=20]
lines=lines[lines['length_hin_sentence']<=20]

In [7]:
# select relevalnt rows
lines = lines.sample(15000 + 752)

In [8]:
lines.describe()

,length_eng_sentence,length_hin_sentence
count,15752.000000,15752.000000
mean,3.966798,6.066404
std,3.308965,3.357732
min,1.000000,3.000000
25%,2.000000,4.000000
50%,3.000000,5.000000
75%,5.000000,7.000000
max,20.000000,20.000000


In [9]:
# compute vocabulary 
all_eng_words=set()
for eng in lines['english']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in lines['hindi']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [10]:
len(all_eng_words), len(all_hindi_words)

(4123, 5042)

In [11]:
# compute sequence length
max_length_src=max(lines['length_hin_sentence'])
max_length_tar=max(lines['length_eng_sentence'])

In [12]:
max_length_src, max_length_tar

(20, 20)

In [13]:
# sort the words
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(4123, 5042)

In [14]:
# add zero padding for encoder and decoder tokens and prepare a index lookkup and reverse lookup
num_encoder_tokens += 1
num_decoder_tokens += 1 

input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [15]:
lines = shuffle(lines)

In [16]:
# make train test split as asked in assignment
X, y = lines['english'], lines['hindi']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 752,random_state=2023)
X_train.shape, X_test.shape

((15000,), (752,))

In [17]:
# X_train = X_train.append(X_test)
# X_train.shape

In [18]:
# store in case og future use
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [19]:
# genetates batch with giving encoder input, decoder input and decoder target needed for training/testing the model
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] 
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] 
                    if t>0:
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [20]:
# create a model using keras and tensorflow
latent_dim=450

encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

2023-11-18 16:11:41.860002: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-11-18 16:11:41.860024: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-11-18 16:11:41.860037: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (sharchit-Ku): /proc/driver/nvidia/version does not exist
2023-11-18 16:11:41.860235: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [21]:
# compile said model and use rmsprop as optimizer and log loss as loss function
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [22]:
# print the summary of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 450)    1855800     ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 450)    2269350     ['input_2[0][0]']                
                                                                                              

In [23]:
# hyper parameters
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 135

In [24]:
# fit the models
model.fit_generator(generator = generate_batch(X_train, y_train,    batch_size = batch_size),
steps_per_epoch = train_samples//batch_size,
epochs=epochs,
validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
validation_steps = val_samples//batch_size)

Epoch 1/135


/tmp/ipykernel_85240/1894032023.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator = generate_batch(X_train, y_train,    batch_size = batch_size),


117/117 [==============================] - 77s 619ms/step - loss: 6.3301 - val_loss: 5.6451
Epoch 2/135
117/117 [==============================] - 69s 586ms/step - loss: 5.5800 - val_loss: 5.4931
Epoch 3/135
117/117 [==============================] - 69s 589ms/step - loss: 5.4250 - val_loss: 5.3901
Epoch 4/135
117/117 [==============================] - 68s 583ms/step - loss: 5.2966 - val_loss: 5.2966
Epoch 5/135
117/117 [==============================] - 69s 587ms/step - loss: 5.1698 - val_loss: 5.1828
Epoch 6/135
117/117 [==============================] - 68s 584ms/step - loss: 5.0583 - val_loss: 5.0742
Epoch 7/135
117/117 [==============================] - 68s 584ms/step - loss: 4.9292 - val_loss: 4.9604
Epoch 8/135
117/117 [==============================] - 68s 584ms/step - loss: 4.7908 - val_loss: 4.8138
Epoch 9/135
117/117 [==============================] - 68s 585ms/step - loss: 4.6303 - val_loss: 4.6776
Epoch 10/135
117/117 [==============================] - 68s 585ms/step - los

In [25]:
# save the models
model.save_weights('eng_hin.h5')

In [26]:
# align the model for predictions
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [27]:
# function that given a encoded sequence gives the hindi output
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1,1))

    target_seq[0, 0] = target_token_index['<sos>']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        if (sampled_char == '<eos>' or
           len(decoded_sentence) > 50):
            stop_condition = True
        
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        states_value = [h, c]

    return decoded_sentence

In [28]:
# create batch
train_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [29]:
X_test.shape

(752,)

In [30]:
# data structure for storing result
log = {'english': [], 'actual_hindi': [], 'predicted_hindi': []}

In [31]:
from tqdm import trange

In [32]:
# predict the results
for i in trange(752):
    k+=1
    (input_seq, actual_output), _ = next(train_gen)
    decoded_sentence = decode_sequence(input_seq)
    log['english'].append(X_train[k:k+1].values[0])
    log['actual_hindi'].append(y_train[k:k+1].values[0][5:-5])
    log['predicted_hindi'].append(decoded_sentence[:-5])

  0%|          | 0/752 [00:00<?, ?it/s]

1/1 [==============================] - 0s 15ms/step


  0%|          | 1/752 [00:02<25:05,  2.00s/it]

1/1 [==============================] - 0s 16ms/step


  0%|          | 2/752 [00:02<11:12,  1.11it/s]

...

100%|█████████▉| 750/752 [03:50<00:00,  3.22it/s]

1/1 [==============================] - 0s 19ms/step


100%|█████████▉| 751/752 [03:51<00:00,  3.42it/s]

1/1 [==============================] - 0s 20ms/step


100%|██████████| 752/752 [03:51<00:00,  3.25it/s]


In [33]:
# save it
df_predicted = pd.DataFrame(log)

In [34]:
df_predicted.to_csv('preds.csv')